In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [3]:
import sys
!{sys.executable} -m pip install tensorflow

    100% |████████████████████████████████| 92.5MB 13kB/s eta 0:00:011    33% |██████████▋                     | 30.8MB 2.6MB/s eta 0:00:24    60% |███████████████████▌            | 56.4MB 688kB/s eta 0:00:53    81% |██████████████████████████      | 75.3MB 1.2MB/s eta 0:00:14    90% |████████████████████████████▉   | 83.3MB 598kB/s eta 0:00:16    92% |█████████████████████████████▊  | 86.0MB 863kB/s eta 0:00:08
  Using cached https://files.pythonhosted.org/packages/90/85/64c82949765cfb246bbdaf5aca2d55f400f792655927a017710a78445def/Keras_Applications-1.0.7-py2.py3-none-any.whl
    100% |████████████████████████████████| 102kB 2.4MB/s a 0:00:011
  Using cached https://files.pythonhosted.org/packages/35/d5/4f8410ac303e690144f0a0603c4b8fd3b986feb2749c435f7cdbb288f17e/numpy-1.16.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
    100% |███████████████

In [2]:

# load the dataset
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [3]:
print(trainDF)

                                                   text       label
0     Stuning even for the non-gamer: This sound tra...  __label__2
1     The best soundtrack ever to anything.: I'm rea...  __label__2
2     Amazing!: This soundtrack is my favorite music...  __label__2
3     Excellent Soundtrack: I truly like this soundt...  __label__2
4     Remember, Pull Your Jaw Off The Floor After He...  __label__2
5     an absolute masterpiece: I am quite sure any o...  __label__2
6     Buyer beware: This is a self-published book, a...  __label__1
7     Glorious story: I loved Whisper of the wicked ...  __label__2
8     A FIVE STAR BOOK: I just finished reading Whis...  __label__2
9     Whispers of the Wicked Saints: This was a easy...  __label__2
10    The Worst!: A complete waste of time. Typograp...  __label__1
11    Great book: This was a great book,I just could...  __label__2
12    Great Read: I thought this book was brilliant,...  __label__2
13    Oh please: I guess you have to be a romanc

In [4]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

In [5]:
print(valid_y)

3543    __label__1
6324    __label__2
7951    __label__2
7986    __label__2
2066    __label__2
1716    __label__2
4311    __label__1
5951    __label__2
8159    __label__1
1814    __label__2
9898    __label__1
5922    __label__2
1582    __label__1
2855    __label__2
8710    __label__2
5726    __label__1
5734    __label__2
8157    __label__2
5665    __label__1
5033    __label__2
3027    __label__1
1685    __label__1
1988    __label__2
8464    __label__2
3110    __label__1
4691    __label__2
8347    __label__2
4495    __label__1
3962    __label__1
828     __label__2
           ...    
8582    __label__2
5744    __label__2
324     __label__2
5370    __label__1
9633    __label__2
3602    __label__2
9955    __label__1
9887    __label__1
6072    __label__2
6357    __label__1
694     __label__1
444     __label__1
6391    __label__1
5825    __label__1
8032    __label__1
9051    __label__2
8689    __label__2
3721    __label__2
6428    __label__1
5409    __label__1
8255    __label__2
4809    __la

In [6]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [7]:
print(valid_y)

[0 1 1 ... 0 1 0]


In [8]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [9]:
print(xtrain_count)

  (0, 26)	1
  (0, 710)	5
  (0, 989)	1
  (0, 1052)	1
  (0, 1601)	1
  (0, 1743)	2
  (0, 1750)	1
  (0, 2016)	1
  (0, 2017)	1
  (0, 2136)	1
  (0, 5094)	1
  (0, 5849)	1
  (0, 6480)	2
  (0, 6492)	1
  (0, 7469)	1
  (0, 7569)	1
  (0, 8143)	1
  (0, 8748)	1
  (0, 8899)	1
  (0, 9918)	1
  (0, 10713)	1
  (0, 11591)	1
  (0, 12908)	1
  (0, 13224)	2
  (0, 14030)	1
  :	:
  (7499, 25027)	1
  (7499, 26107)	1
  (7499, 27417)	1
  (7499, 27765)	1
  (7499, 28020)	1
  (7499, 28076)	1
  (7499, 28082)	5
  (7499, 28106)	2
  (7499, 28121)	1
  (7499, 28173)	1
  (7499, 28224)	2
  (7499, 28595)	1
  (7499, 28984)	1
  (7499, 29618)	1
  (7499, 29787)	1
  (7499, 29966)	1
  (7499, 30106)	1
  (7499, 30299)	1
  (7499, 30831)	1
  (7499, 30851)	2
  (7499, 30965)	1
  (7499, 30976)	1
  (7499, 31096)	1
  (7499, 31102)	1
  (7499, 31514)	1


In [10]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [11]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [12]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)

/home/saksham/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
print(accuracy)

0.8656


In [14]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",accuracy)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
7500/7500 [==============================] - 4s 584us/step - loss: 0.5259
NN, Ngram Level TF IDF Vectors 0.502


In [28]:
#tfidf_vect_ngram.get_feature_names()
#tfidf_vect_ngram.vocabulary_
print(xtrain_tfidf_ngram)

  (0, 4537)	0.2789661404512955
  (0, 4196)	0.1726723233871585
  (0, 4106)	0.2124571711287434
  (0, 4101)	0.13931350150767105
  (0, 3977)	0.20942041430555516
  (0, 3974)	0.39686672017404606
  (0, 3485)	0.2625990198548298
  (0, 2785)	0.08013728971859356
  (0, 2553)	0.19335001928649403
  (0, 2524)	0.1723991628577871
  (0, 2283)	0.1852552298785334
  (0, 1762)	0.16456634559447525
  (0, 1484)	0.2499613220959175
  (0, 978)	0.1389207931494489
  (0, 455)	0.25970840876948037
  (0, 432)	0.2043140011981171
  (0, 220)	0.23437627697616004
  (0, 135)	0.42031379373541894
  (1, 4875)	0.21631330463090784
  (1, 4873)	0.3481491322004817
  (1, 4780)	0.1390625217822411
  (1, 4678)	0.23936704004410636
  (1, 4516)	0.2291105170565627
  (1, 3913)	0.20605678164336422
  (1, 3681)	0.13256234183552104
  :	:
  (7499, 4072)	0.16837531471554526
  (7499, 3985)	0.14708012407399546
  (7499, 3655)	0.24056046291141128
  (7499, 3539)	0.15932084038561214
  (7499, 3264)	0.2206124780250366
  (7499, 2841)	0.14782990782845454
  